In [1]:
# %%capture
# This cell will take time
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install optuna

Found existing installation: unsloth 2024.11.7
Uninstalling unsloth-2024.11.7:
  Successfully uninstalled unsloth-2024.11.7
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-wgf492tj/unsloth_0677ae4c88044b5faa2430c98cd598af
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-wgf492tj/unsloth_0677ae4c88044b5faa2430c98cd598af
  Resolved https://github.com/unslothai/unsloth.git to commit f26d4e739ed507de7a9088da53d10fd02f58d160
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.11.7-py3-none-any.whl size=163138 sha256=4acd00b646a1d5c4c9ae849a1cb66c1624ed0550f9f4ea3c9a48a5e4c52a95e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-kdq2575n/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [2]:
from google.colab import drive
drive.mount('/content/drive')

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
# Load model and wrap with LoRA

model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
# download and load competition dataset

from datasets import load_dataset
from datasets import Dataset
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")

In [5]:
# split


train_test_split = 0.80
train_dataset_size = int(len(dataset['train']) * train_test_split)
test_dataset_size = len(dataset['train']) - train_dataset_size

train_dataset, test_dataset = Dataset.from_dict(dataset['train'][:train_dataset_size]), Dataset.from_dict(dataset['train'][train_dataset_size:])


In [6]:
prompt = """You are a great mathematician and you are tasked with finding if an answer to a given maths question is correct or not. Yout response should be 'True' if correct, otherwise 'False'. Below is Question and Answer.


### Question:
{}

### Answer:
{}

### Solution:

### Output:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["question"]
    ans       = examples["answer"]
    expln     = examples["solution"]
    output    = examples["is_correct"]
    texts = []
    for instruction, input, expln, output in zip(question, ans, expln, output):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [7]:
train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
test_dataset = test_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/800000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

In [8]:
train_tmp = Dataset.from_dict(train_dataset[:5000])
test_tmp = Dataset.from_dict(test_dataset[:500])

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import gc

def objective(trial):
    # Define hyperparameters to tune
    training_args = TrainingArguments(
        per_device_train_batch_size = trial.suggest_int('batch_size', 1, 4),
        gradient_accumulation_steps = trial.suggest_int('gradient_accumulation_steps', 2, 8),
        warmup_steps = trial.suggest_int('warmup_steps', 1, 10),
        num_train_epochs = 1,  # Keep fixed for consistency
        max_steps = trial.suggest_int('max_steps', 5, 20),
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True),
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = trial.suggest_float('weight_decay', 0.001, 0.1, log=True),
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = f"outputs/trial_{trial.number}",
        report_to = "none",
    )

    # Create trainer with the trial hyperparameters
    trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = train_tmp,
        eval_dataset = test_tmp,
        dataset_text_field = "text",
        max_seq_length = max_seq_length,
        dataset_num_proc = 4,
        packing = False,
        args = training_args
    )

    # Train the model
    trainer_stats = trainer.train()

    # Evaluate on test set
    test_accuracy = trainer.evaluate()


    torch.cuda.empty_cache()
    gc.collect
    return test_accuracy['eval_loss']

In [12]:
import optuna

# Create the study
study = optuna.create_study(direction="minimize")  # minimize loss
n_trials = 5  # Number of trials to run

# Run the optimization
study.optimize(objective, n_trials=n_trials)

# Print the results
print("Best trial:")
print("  Value: ", study.best_trial.value)
print("  Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

[I 2024-11-15 07:02:40,166] A new study created in memory with name: no-name-3de66cd4-b260-48da-9459-6b04e8da7a88


Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 3
\        /    Total batch size = 3 | Total steps = 6
 "-____-"     Number of trainable parameters = 335,544,320


Step,Training Loss
1,1.368300
2,1.615700
3,1.282300
4,1.441600
5,1.140000
6,0.939700


...

Map (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 3 | Gradient Accumulation steps = 6
\        /    Total batch size = 18 | Total steps = 14
 "-____-"     Number of trainable parameters = 335,544,320


Step,Training Loss
1,0.465400
2,0.606900
3,0.606000
4,0.542400
5,0.592500
6,0.717300
7,0.696400
8,0.802400
9,0.741000
10,0.865200


[I 2024-11-15 07:20:47,079] Trial 4 finished with value: 0.7997326850891113 and parameters: {'batch_size': 3, 'gradient_accumulation_steps': 6, 'warmup_steps': 6, 'max_steps': 14, 'learning_rate': 0.00044976927936280013, 'weight_decay': 0.01372324870476428}. Best is trial 3 with value: 0.7892608642578125.


Best trial:
  Value:  0.7892608642578125
  Params: 
    batch_size: 2
    gradient_accumulation_steps: 5
    warmup_steps: 2
    max_steps: 18
    learning_rate: 0.0009440256243209805
    weight_decay: 0.07134953536188349


In [13]:
# training with best trial

training_args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 5,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 18,
        learning_rate = 0.0009440256243209805,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.07134953536188349,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args
)

Map (num_proc=4):   0%|          | 0/800000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 800,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 5
\        /    Total batch size = 10 | Total steps = 18
 "-____-"     Number of trainable parameters = 335,544,320


Step,Training Loss
1,0.804900
2,0.871500
3,0.860800
4,0.825600
5,0.907000
6,0.713700
7,0.662900
8,0.851000
9,0.661300
10,0.881800


In [15]:
# save to google drive

model.save_pretrained("drive/MyDrive/Colab Notebooks/lora_model_3") # replace with your disire path
tokenizer.save_pretrained("drive/MyDrive/Colab Notebooks/lora_model_3")

('drive/MyDrive/Colab Notebooks/lora_model_3/tokenizer_config.json',
 'drive/MyDrive/Colab Notebooks/lora_model_3/special_tokens_map.json',
 'drive/MyDrive/Colab Notebooks/lora_model_3/tokenizer.json')

Inference

In [16]:
from transformers import TextGenerationPipeline

class MyPipeline(TextGenerationPipeline):
    def postprocess(pipeline ,dict):
      # dict_keys(['generated_sequence', 'input_ids', 'prompt_text'])
      tmp = dict['generated_sequence']
      holder = len(dict['input_ids'][0])
      tmp2 = tokenizer.batch_decode([tmp[0][0][holder:]], skip_special_tokens=True)
      return tmp2[0]

FastLanguageModel.for_inference(model)
pipe = MyPipeline(
    task="text-generation",
    model = model,
    tokenizer = tokenizer,
    batch_size = 4,
    eos_token_id = model.config.eos_token_id,
    max_new_tokens = 64
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausa

In [17]:


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["question"]
    ans       = examples["answer"]
    texts = []
    for instruction, input in zip(question, ans):
        text = prompt.format(instruction, input, "", "")
        texts.append(text)
    return { "text" : texts, }

test_format = test_dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

In [18]:
test_input = test_format['text']
test_key = test_dataset['is_correct']

In [30]:
res = []

for i in range(10):
  # sperate into 10 runs
  l = 100*i
  r = 100*(i+1)
  print(l, r)
  res += pipe(test_input[l:r])

0 100
100 200
200 300
300 400
400 500
500 600
600 700
700 800
800 900
900 1000


In [31]:
res_tmp = []

for i,v in enumerate(res):
  tf = v.split('\n')[0]
  if tf == 'True':
    res_tmp.append(True)
  else:
    res_tmp.append(False)

In [32]:
correct = 0
incorrect = 0
for i in range(len(res_tmp)):
  if test_key[i] == res_tmp[i]:
    correct += 1
  elif test_key[i] != res_tmp[i]:
    incorrect += 1
  else:
    print(i)

print(correct/len(res_tmp))

0.607
